# 영상 받아서 이미지 한장씩 저장하기

In [2]:
import cv2
import os

filepath = './video1.mp4'
video = cv2.cv2.VideoCapture(filepath)

if not video.isOpened():
    print("Could not Open :",filepath)
    exit(0)
    
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 1553
width : 1920
height : 1080
fps : 30.0


In [3]:
new_path = './new_image/' # 이미지가 담길 폴더
import cv2
vidcap = cv2.VideoCapture(filepath)  # 영상이 있는 경로 
count = 0 

while(vidcap.isOpened()): 
    ret, image = vidcap.read() # 이미지 사이즈 960x540으로 변경 
    if ret:
        # image = cv2.resize(image, (960, 540)) # 30프레임당 하나씩 이미지 추출 
        if(int(vidcap.get(1)) % 30 == 0): 
            print('Saved frame number : ' + str(int(vidcap.get(1)))) # 추출된 이미지가 저장되는 경로 
            cv2.imwrite(new_path+"frame%d.png" % count, image) #
            print('Saved frame%d.jpg' % count) 
            count += 1
    else:
        break
        
vidcap.release()

Saved frame number : 30
Saved frame0.jpg
Saved frame number : 60
Saved frame1.jpg
Saved frame number : 90
Saved frame2.jpg
Saved frame number : 120
Saved frame3.jpg
Saved frame number : 150
Saved frame4.jpg
Saved frame number : 180
Saved frame5.jpg
Saved frame number : 210
Saved frame6.jpg
Saved frame number : 240
Saved frame7.jpg
Saved frame number : 270
Saved frame8.jpg
Saved frame number : 300
Saved frame9.jpg
Saved frame number : 330
Saved frame10.jpg
Saved frame number : 360
Saved frame11.jpg
Saved frame number : 390
Saved frame12.jpg
Saved frame number : 420
Saved frame13.jpg
Saved frame number : 450
Saved frame14.jpg
Saved frame number : 480
Saved frame15.jpg
Saved frame number : 510
Saved frame16.jpg
Saved frame number : 540
Saved frame17.jpg
Saved frame number : 570
Saved frame18.jpg
Saved frame number : 600
Saved frame19.jpg
Saved frame number : 630
Saved frame20.jpg
Saved frame number : 660
Saved frame21.jpg
Saved frame number : 690
Saved frame22.jpg
Saved frame number : 720

# 연속된 이미지 불러와서 검출 -> 분류 -> 시각화

- 연속된 이미지 파일명 : 20200602_1.jpg (이런식으로 파일명을 변경함) 
- 여기서 검출된 신호등 20200602_1_1, 20200602_1_2 ... 이런식으로

In [4]:
import cv2
import matplotlib.pyplot as plt

## green yellow red 모델들

# 파일명 시작 숫자 설정
seq = 1 # 채번 변수 통일 20220308 by yjlim

# 추출 이미지 저장 위치 설정
# 이미지 한장으로 테스트 하는 거라서 디렉토리 분류 할 필요 없이 한 폴더에 다 담기
path = './vi_data2/'

first_detect = []

##### 1. green #####
def green(img, name, X, Y, W, H):
    global seq, path_g, path
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    name = name.split('.')
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()    

    ## 초록색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (43, 40, 120)
    up = (95, 255, 255)

    # 범위내의 픽셀들은 흰색, 나머지 검은색
    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    
    # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion2 = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)    
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)
    

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))
            
    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - 2 * length - 2 * interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,\
                        X + a[0] - 2 * length - 2 * interval_x:X + a[0] + length + interval_x]
                print(img + '///G' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y, a[1] + a[3] + interval_y, X + a[0] - 2 * length - 2 * interval_x, X + a[0] + length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 2. yellow #####
def yellow(img, name, X, Y, W, H):
    global seq, path_y, path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()

    ## 노랑색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (12, 10, 110)
    up = (31, 255, 255)
    # low = (20, 20, 100)
    # up = (30, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    
    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - length - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - length - interval_x:X + a[0] + 2 * length + interval_x]
                
                print(img + '///Y' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y,X + a[0] - length - interval_x, X + a[0] + 2 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 3. red #####
def red(img, name, X, Y, W, H):
    global seq, path_r,path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 빨강색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (0, 30, 50)
    up = (10, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    # 색 영역 두번째 추가
    low3 = (150, 40, 50)
    up3 = (180, 255, 255)
    
    roi3_mask = cv2.inRange(roi2_hsv, low3, up3)
    roi3_result = cv2.bitwise_and(roi2, roi2, mask = roi3_mask)
    
    # 색 영역 1 + 색 영역 2  
    result_red = roi2_result + roi3_result

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(result_red, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(result_red, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3)
            interval_y = int(a[3] / 4)
            
            if a[1] - interval_y > 0 and X + a[0] - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - interval_x:X + a[0] + 3 * length + interval_x]
                print(img + '///R' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y, X + a[0] - interval_x, X + a[0] + 3 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] +'_'+ str(seq) + '.jpg', real3)
                seq = seq + 1

In [5]:
import sys
import glob
import cv2

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
# 연속된 이미지가 담긴 폴더
#img_files = glob.glob('./video/*.jpg')
img_files = glob.glob('.\\new_image\\*.png')
img_files.sort() # 순서대로 정렬

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
    
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
    if cv2.waitKey(1000) == 27:     
        break

    index += 1      
    
    # ROI 지정, x,y,w,h ,N 설정 완료
    x1 = 300
    y1 = 0
    w1 = 1400
    h1 = 640
    
    green(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    yellow(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    red(filepath, name, X = x1, Y = y1, W = w1, H = h1)


.\new_image\frame0.png///G1.jpg
.\new_image\frame0.png///G2.jpg
.\new_image\frame0.png///Y3.jpg
.\new_image\frame0.png///Y4.jpg
.\new_image\frame0.png///Y5.jpg
.\new_image\frame0.png///Y6.jpg
.\new_image\frame0.png///Y7.jpg
.\new_image\frame0.png///R8.jpg
.\new_image\frame1.png///G9.jpg
.\new_image\frame1.png///G10.jpg
.\new_image\frame1.png///Y11.jpg
.\new_image\frame1.png///R12.jpg
.\new_image\frame10.png///G13.jpg
.\new_image\frame10.png///G14.jpg
.\new_image\frame11.png///G15.jpg
.\new_image\frame11.png///G16.jpg
.\new_image\frame18.png///Y17.jpg
.\new_image\frame19.png///Y18.jpg
.\new_image\frame20.png///G19.jpg
.\new_image\frame20.png///G20.jpg
.\new_image\frame20.png///G21.jpg
.\new_image\frame20.png///R22.jpg
.\new_image\frame21.png///R23.jpg
.\new_image\frame21.png///R24.jpg
.\new_image\frame21.png///R25.jpg
.\new_image\frame21.png///R26.jpg
.\new_image\frame22.png///R27.jpg
.\new_image\frame22.png///R28.jpg
.\new_image\frame22.png///R29.jpg
.\new_image\frame23.png///Y30.jpg
.

In [6]:
import os
import cv2
import numpy as np
import random
from PIL import Image

from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf

### 신호등 인식 모델

In [7]:
model_detection = tf.keras.models.load_model('traffic_light_detection_v16.h5')

#### 이미지 resize 해서 해당 폴더에 다시 저장

In [8]:
path_detection = 'vi_data2/' # 색 영역으로 검출한 모든 신호등 후보 영역 이미지

file_list_d = os.listdir(path_detection)

if ".DS_Store" in file_list_d:
    file_list_d.remove(".DS_Store")

data_detection = []

for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img_d = cv2.imread(path_detection + A)
    res_d = cv2.resize(img_d, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(path_detection + A, res_d)
    
for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img3 = Image.open(path_detection + A)
    data_detection.append(np.array(img3))
    img3.close()

In [9]:
data_detection = np.asarray(data_detection) / 255.0

In [10]:
prediction = model_detection.predict(data_detection)

In [11]:
light_order = {0 : "negative", 1 : "positive"}

light_detect_img = [] # 파일명, 예측 결과(negative, positive)
data_detection2 = [] # 신호등 이미지

for i, x in enumerate(prediction):
    pred_detect = ""
    
    if x >= 0.85:
        pred_detect = light_order[1]
    else:
        pred_detect = light_order[0]
    
    # 신호등으로 85퍼센트 이상 인식된 이미지만 추출
    if pred_detect == "positive":
        light_detect_img.append([file_list_d[i], light_order[1]])

        imgPath = path_detection + file_list_d[i]

        img_detection = Image.open(imgPath)
        data_detection2.append(np.array(img_detection)) # 신호등으로 인식된 이미지

In [12]:
len(data_detection2)

60

In [13]:
light_detect_img

[['frame0_1.jpg', 'positive'],
 ['frame0_2.jpg', 'positive'],
 ['frame10_13.jpg', 'positive'],
 ['frame10_14.jpg', 'positive'],
 ['frame11_15.jpg', 'positive'],
 ['frame11_16.jpg', 'positive'],
 ['frame1_10.jpg', 'positive'],
 ['frame1_9.jpg', 'positive'],
 ['frame21_24.jpg', 'positive'],
 ['frame22_29.jpg', 'positive'],
 ['frame23_31.jpg', 'positive'],
 ['frame24_35.jpg', 'positive'],
 ['frame25_38.jpg', 'positive'],
 ['frame25_41.jpg', 'positive'],
 ['frame26_46.jpg', 'positive'],
 ['frame26_48.jpg', 'positive'],
 ['frame27_50.jpg', 'positive'],
 ['frame27_52.jpg', 'positive'],
 ['frame28_56.jpg', 'positive'],
 ['frame28_59.jpg', 'positive'],
 ['frame29_66.jpg', 'positive'],
 ['frame29_68.jpg', 'positive'],
 ['frame30_72.jpg', 'positive'],
 ['frame30_76.jpg', 'positive'],
 ['frame31_78.jpg', 'positive'],
 ['frame31_82.jpg', 'positive'],
 ['frame32_85.jpg', 'positive'],
 ['frame32_87.jpg', 'positive'],
 ['frame33_90.jpg', 'positive'],
 ['frame33_92.jpg', 'positive'],
 ['frame34_97.jpg

### 신호등 색 분류

In [14]:
model = tf.keras.models.load_model('traffic_light_0315_2.h5')

In [15]:
# # 검출된 신호등 이미지를 입력 값으로 만들기
# Path2 = 'vi_data2/'
# file_list = os.listdir(Path2)
# file_list.sort()
# if ".DS_Store" in file_list:
#     file_list.remove(".DS_Store")

# data2 = []

# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img2 = cv2.imread(Path2+A)
#         res2 = cv2.resize(img2, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
#         cv2.imwrite(Path2+A,res2)
#     except Exception as e:
#         print(str(e))
    
# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img3 = Image.open(Path2 + A)
#         data2.append(np.array(img3))
#         img3.close()
#     except Exception as e:
#         print(str(e))

In [16]:
# data2

In [17]:
# data2 = np.asarray(data2) / 255.0
data_detection2 = np.asarray(data_detection2) / 255.0
prediction = model.predict(data_detection2)

In [18]:
prediction

array([[9.99957085e-01, 2.33316459e-05, 1.95803077e-05],
       [9.99926925e-01, 4.01254983e-05, 3.29030954e-05],
       [9.99979496e-01, 1.15309012e-05, 8.90431602e-06],
       [9.99980450e-01, 1.10346828e-05, 8.46268267e-06],
       [9.99957681e-01, 2.39376768e-05, 1.83640877e-05],
       [9.99940038e-01, 3.45118788e-05, 2.53377893e-05],
       [9.96749282e-01, 1.79784559e-03, 1.45289244e-03],
       [9.97821927e-01, 1.21571112e-03, 9.62404418e-04],
       [9.73652641e-04, 9.98890460e-01, 1.35807510e-04],
       [1.64743362e-03, 9.98134732e-01, 2.17770154e-04],
       [1.21263647e-03, 9.98730242e-01, 5.71218952e-05],
       [1.77117618e-05, 9.99980569e-01, 1.69583745e-06],
       [1.04031712e-03, 9.98884976e-01, 7.46933292e-05],
       [4.34146523e-06, 9.99995232e-01, 4.90463492e-07],
       [1.42494112e-03, 9.98206139e-01, 3.68931593e-04],
       [1.16918034e-06, 9.99998689e-01, 1.14208255e-07],
       [3.20342579e-03, 9.96085286e-01, 7.11274566e-04],
       [4.81337963e-07, 9.99999

In [19]:
len(first_detect) # 처음 색영역으로 신호등 후보 영역 검출한 개수

208

In [20]:
light_order = {0 : "green", 1 : "red", 2 : "yellow", 3 : "unknown"}

# 신호등 파일명과 색을 한 리스트에 담기 
detect_img = [] 

for i, x in enumerate(prediction):
    count = 0
    for y in x:
        if y > 0.90:    # 90퍼센트 이상이면 인식
            count += 1
            
    if count > 0:
        print(str(i) + "번째 신호등: " + light_detect_img[i][0] + " : " + light_order[np.argmax(x)])
        detect_img.append([light_detect_img[i][0], light_order[np.argmax(x)]])
    else:
        print(str(i)+"번째 신호등: " + light_detect_img[i][0] + " : " + light_order[3])

0번째 신호등: frame0_1.jpg : green
1번째 신호등: frame0_2.jpg : green
2번째 신호등: frame10_13.jpg : green
3번째 신호등: frame10_14.jpg : green
4번째 신호등: frame11_15.jpg : green
5번째 신호등: frame11_16.jpg : green
6번째 신호등: frame1_10.jpg : green
7번째 신호등: frame1_9.jpg : green
8번째 신호등: frame21_24.jpg : red
9번째 신호등: frame22_29.jpg : red
10번째 신호등: frame23_31.jpg : red
11번째 신호등: frame24_35.jpg : red
12번째 신호등: frame25_38.jpg : red
13번째 신호등: frame25_41.jpg : red
14번째 신호등: frame26_46.jpg : red
15번째 신호등: frame26_48.jpg : red
16번째 신호등: frame27_50.jpg : red
17번째 신호등: frame27_52.jpg : red
18번째 신호등: frame28_56.jpg : red
19번째 신호등: frame28_59.jpg : red
20번째 신호등: frame29_66.jpg : red
21번째 신호등: frame29_68.jpg : red
22번째 신호등: frame30_72.jpg : red
23번째 신호등: frame30_76.jpg : red
24번째 신호등: frame31_78.jpg : red
25번째 신호등: frame31_82.jpg : red
26번째 신호등: frame32_85.jpg : red
27번째 신호등: frame32_87.jpg : red
28번째 신호등: frame33_90.jpg : red
29번째 신호등: frame33_92.jpg : red
30번째 신호등: frame34_97.jpg : red
31번째 신호등: frame34_99.jpg : red
32번째 신호등:

In [21]:
detect_img

[['frame0_1.jpg', 'green'],
 ['frame0_2.jpg', 'green'],
 ['frame10_13.jpg', 'green'],
 ['frame10_14.jpg', 'green'],
 ['frame11_15.jpg', 'green'],
 ['frame11_16.jpg', 'green'],
 ['frame1_10.jpg', 'green'],
 ['frame1_9.jpg', 'green'],
 ['frame21_24.jpg', 'red'],
 ['frame22_29.jpg', 'red'],
 ['frame23_31.jpg', 'red'],
 ['frame24_35.jpg', 'red'],
 ['frame25_38.jpg', 'red'],
 ['frame25_41.jpg', 'red'],
 ['frame26_46.jpg', 'red'],
 ['frame26_48.jpg', 'red'],
 ['frame27_50.jpg', 'red'],
 ['frame27_52.jpg', 'red'],
 ['frame28_56.jpg', 'red'],
 ['frame28_59.jpg', 'red'],
 ['frame29_66.jpg', 'red'],
 ['frame29_68.jpg', 'red'],
 ['frame30_72.jpg', 'red'],
 ['frame30_76.jpg', 'red'],
 ['frame31_78.jpg', 'red'],
 ['frame31_82.jpg', 'red'],
 ['frame32_85.jpg', 'red'],
 ['frame32_87.jpg', 'red'],
 ['frame33_90.jpg', 'red'],
 ['frame33_92.jpg', 'red'],
 ['frame34_97.jpg', 'red'],
 ['frame34_99.jpg', 'red'],
 ['frame35_103.jpg', 'red'],
 ['frame35_106.jpg', 'red'],
 ['frame36_110.jpg', 'red'],
 ['frame

In [22]:
print(len(file_list_d),len(detect_img)) # 모델에 적용한 이미지, 이미지 검출 코드에서 나온거

208 60


In [23]:
xy = []
for i in range(0, len(detect_img)):
    for j in range(0, len(first_detect)):

        if first_detect[j][1] == detect_img[i][0].split(".")[0]:
            xy.append([first_detect[j][1], first_detect[j][0], detect_img[i][1]])
    #print(detect_img[i][0].split(".")[0])

In [24]:
xy

[['frame0_1', [542, 562, 795, 849], 'green'],
 ['frame0_2', [530, 550, 1011, 1074], 'green'],
 ['frame10_13', [481, 507, 737, 809], 'green'],
 ['frame10_14', [472, 497, 951, 1023], 'green'],
 ['frame11_15', [44, 100, 453, 603], 'green'],
 ['frame11_16', [22, 76, 961, 1111], 'green'],
 ['frame1_10', [297, 335, 1017, 1119], 'green'],
 ['frame1_9', [324, 354, 607, 694], 'green'],
 ['frame21_24', [611, 637, 931, 997], 'red'],
 ['frame22_29', [582, 612, 911, 983], 'red'],
 ['frame23_31', [549, 582, 892, 975], 'red'],
 ['frame24_35', [493, 529, 865, 953], 'red'],
 ['frame25_38', [628, 642, 942, 1000], 'red'],
 ['frame25_41', [427, 463, 841, 935], 'red'],
 ['frame26_46', [613, 639, 933, 1005], 'red'],
 ['frame26_48', [350, 394, 809, 919], 'red'],
 ['frame27_50', [600, 630, 929, 1001], 'red'],
 ['frame27_52', [271, 319, 775, 907], 'red'],
 ['frame28_56', [595, 621, 929, 995], 'red'],
 ['frame28_59', [211, 261, 757, 889], 'red'],
 ['frame29_66', [593, 619, 931, 997], 'red'],
 ['frame29_68', [16

In [25]:
array = []
for i in range(0, len(xy)):
    coor = xy[i][1]
    color = xy[i][2]
    for j in range(i+1, len(xy)):
        if xy[i][0] == xy[j][0]:
            coor = xy[j][1]
            color = xy[j][2]
    if [xy[i][0], coor, color] not in array:
        array.append([xy[i][0], coor, color])
    #print(xy[i])

In [26]:
array

[['frame0_1', [542, 562, 795, 849], 'green'],
 ['frame0_2', [530, 550, 1011, 1074], 'green'],
 ['frame10_13', [481, 507, 737, 809], 'green'],
 ['frame10_14', [472, 497, 951, 1023], 'green'],
 ['frame11_15', [44, 100, 453, 603], 'green'],
 ['frame11_16', [22, 76, 961, 1111], 'green'],
 ['frame1_10', [297, 335, 1017, 1119], 'green'],
 ['frame1_9', [324, 354, 607, 694], 'green'],
 ['frame21_24', [611, 637, 931, 997], 'red'],
 ['frame22_29', [582, 612, 911, 983], 'red'],
 ['frame23_31', [549, 582, 892, 975], 'red'],
 ['frame24_35', [493, 529, 865, 953], 'red'],
 ['frame25_38', [628, 642, 942, 1000], 'red'],
 ['frame25_41', [427, 463, 841, 935], 'red'],
 ['frame26_46', [613, 639, 933, 1005], 'red'],
 ['frame26_48', [350, 394, 809, 919], 'red'],
 ['frame27_50', [600, 630, 929, 1001], 'red'],
 ['frame27_52', [271, 319, 775, 907], 'red'],
 ['frame28_56', [595, 621, 929, 995], 'red'],
 ['frame28_59', [211, 261, 757, 889], 'red'],
 ['frame29_66', [593, 619, 931, 997], 'red'],
 ['frame29_68', [16

# array에 담긴 파일명으로 원천 이미지에 시각화

- path 경로에서 원천 이미지 찾아서 그 이미지에다가 rectangle 그리는 거여서 덮어씌워짐  
- 덮어씌워지니까 원천 이미지는 꼭 백업해둘것

In [27]:
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0] + '.png'
    print(path,imgname)

frame0_1 [542, 562, 795, 849] green
./new_image/ frame0.png
frame0_2 [530, 550, 1011, 1074] green
./new_image/ frame0.png
frame10_13 [481, 507, 737, 809] green
./new_image/ frame10.png
frame10_14 [472, 497, 951, 1023] green
./new_image/ frame10.png
frame11_15 [44, 100, 453, 603] green
./new_image/ frame11.png
frame11_16 [22, 76, 961, 1111] green
./new_image/ frame11.png
frame1_10 [297, 335, 1017, 1119] green
./new_image/ frame1.png
frame1_9 [324, 354, 607, 694] green
./new_image/ frame1.png
frame21_24 [611, 637, 931, 997] red
./new_image/ frame21.png
frame22_29 [582, 612, 911, 983] red
./new_image/ frame22.png
frame23_31 [549, 582, 892, 975] red
./new_image/ frame23.png
frame24_35 [493, 529, 865, 953] red
./new_image/ frame24.png
frame25_38 [628, 642, 942, 1000] red
./new_image/ frame25.png
frame25_41 [427, 463, 841, 935] red
./new_image/ frame25.png
frame26_46 [613, 639, 933, 1005] red
./new_image/ frame26.png
frame26_48 [350, 394, 809, 919] red
./new_image/ frame26.png
frame27_50 [60

In [28]:
# 원천이미지 있는 디렉토리 경로
# 해당 디렉토리 백업해두기
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 


for i in range(len(array)):
    a, b, c = array[i]
    
    frame = a.split('_')[0]
    imgname = frame + '.png'
    
    for j in range(i + 1, len(array)):
        a1, b1, c1 = array[j]
        a1 = a1.split('_')[0]
        
        if frame == a1: # 같은 이미지 내에서만 비교
            if c == 'red' or c == 'yellow':
                if c1 == 'red' or c1 == 'yellow':
                    if abs(b[2] - b1[2]) <= 30:
                        # 비슷한 영역이 두번 검출되었으므로 객체 하나 삭제(yellow 삭제)
                        if c == 'red' or c == c1:
                            array[j][1] = [0, 0, 0, 0]
                        else:
                            b = 0, 0, 0, 0
        else:
            break

            
    img = cv2.imread(path + imgname)
    
    y, h, x, w = b
    w = w - x
    h = h - y
    text = c 
    org = (x, y - 8)
    
    if x > 0 and y > 0 and w > 0 and h > 0:
        cv2.rectangle(img, (x, y, w, h), (139, 255, 224), 2)
        cv2.putText(img, text, org, font, 0.7, (139, 255, 224), 2)
        
    # 기존 원천 이미지에 덮어쓰기 되는거니까 주의!!!
    cv2.imwrite(path + imgname, img)
    

# 원천 이미지 연속 출력 (검출된 내용 반영)

#### 숫자 정렬용 라이브러리 설치

In [29]:
# !pip install natsort

In [30]:
import sys
import glob
import cv2

import natsort # 숫자 정렬용 라이브러리

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
img_files = glob.glob('.\\new_image\\*.png')
# img_files.sort()

# 문자열로 인식해서 [1, 10, ..., 2] 이런식으로 나오는 오류
# natsort 사용하여 [1, 2, ..., 10] 순으로 나올 수 있도록 정렬
img_files = natsort.natsorted(img_files)

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
	# ESC가 입력되면 break
    cv2.imshow('video_test', img)
    if cv2.waitKey(1000) == 27:     
        break

	# index가 이미지 리스트보다 커지거나 같아지면 다시 0으로
    index += 1      
    
    #if index >= count :
        #index = 0 

cv2.waitKey(0)    
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 동영상으로 저장

In [31]:
import numpy as np
import glob
 
img_array = []

filename_list = glob.glob('.\\new_image\\*.png')
filename_list = natsort.natsorted(filename_list)

for filename in filename_list:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('video_test.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()